In [12]:
#=
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid CAilleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the “Software”), to deal in the Software without restriction, including without 
limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the 
Software, and to permit persons to whom the Software is furnished to do so, subject to the following 
conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions 
of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED 
TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL 
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE.

=#

#=

Julia Version 1.8.1 (2022-09-06)

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
I love Python and Julia
Julia is faster for some stuff
Implement Sterne Broco in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

# 1 Packages
using BenchmarkTools
using Test

# 2 Structs

mutable struct RationalNode
    numerator::Int64
    denominator::Int64
    place::Int64
    layer::Int64
end

# 3 Config

function get_depth()
    depth::Int64 = 1
    return depth
end

# 4 model

function count_nodes(depth)
    node_count::Int64 = (2^depth)-1 
    return node_count
end

function init_tree(node_count, this_power)
    zero_over_one   = RationalNode(0,1,0,           0)
    one_over_zero   = RationalNode(1,0,node_count+1,0)
    triangular_part = [RationalNode(0,0,i,0) for i in 1:node_count]
    tree            = [zero_over_one]
    tree = vcat(tree,triangular_part)
    tree = vcat(tree,one_over_zero)
    left_parent         = tree[this_power+this_power+1]
    right_parent        = tree[this_power-this_power+1]
    child_place         = this_power
    child_numerator     = left_parent.numerator   + right_parent.numerator
    child_denominator   = left_parent.denominator + right_parent.denominator
    child               = RationalNode(child_numerator,child_denominator,child_place,1)
    tree[child.place+1] = child
    child_places        = [child.place]
    return tree, child_places
end

function build_tree(depth::Int64)
    node_count::Int64       = count_nodes(depth)
    powers::Array{Int64}    = [2^x for x in (depth-1):-1:0]
    powers = vcat(powers, 0)
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power) 
    child_actual_indices    = [x+1 for x in child_places]
    for current_layer in 2:depth
        this_power = powers[current_layer]    
        grand_child_indices::Array{Int64}  = []
        for parent_place in child_actual_indices
            left_child_index::Int64    =  parent_place - this_power
            right_child_index::Int64   =  parent_place + this_power
            append!(grand_child_indices, [left_child_index, right_child_index])
        end
        child_actual_indices = grand_child_indices
        for this_child_index in child_actual_indices
            this_child          = tree[this_child_index]
            left_parent_index   = this_child_index - this_power
            right_parent_index  = this_child_index + this_power
            left_parent         = tree[left_parent_index]
            right_parent        = tree[right_parent_index]
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            this_child          = RationalNode(child_numerator,child_denominator,this_child.place,current_layer)
            tree[this_child_index] = this_child
        end
    end
    return tree
end

# 5 view

function draw_tree(tree)
    depth       = get_depth()
    over_power  = 2^depth
    height      = 1+depth
    width       = 1+over_power
    numerator_host   = [zeros(width) for _ in 1:height]
    denominator_host = [zeros(width) for _ in 1:height]
    num_printable = ""
    den_printable = ""
    for node in tree
        numerator_host[node.layer+1][node.place+1]  = node.numerator
        denominator_host[node.layer+1][node.place+1]= node.denominator
    end
    all_num = []
    all_den = []
    for num_row in enumerate(numerator_host)
        both = zip(num_row[2],denominator_host[num_row[1]])
        num_printable = ""
        den_printable = ""
        for x in both
            if x == (0,0)
                num_printable = join(num_printable*(" "))
                den_printable = join(den_printable*(" "))
            else
                num_printable = join(num_printable*(string(trunc(Int64,x[1]))))
                den_printable = join(den_printable*(string(trunc(Int64,x[2]))))
            end
        end
        push!(all_num, num_printable)
        push!(all_den, den_printable)
    end
    for num_row in enumerate(all_num)
        num_fig = num_row[2]
        den_fig = all_den[num_row[1]]
        println(num_fig)
        println(den_fig)
    end
end
    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    draw_tree(tree)
    for i in tree
        println("place\t:",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

0 1
1 0
 1 
 1 
place	:0	rational:	0//1
place	:1	rational:	1//1
place	:2	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 1 layers.
  0.000002 seconds
elapsed time (ns):  1590
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [13]:

# 3 Config

function get_depth()
    depth::Int64 = 2
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    draw_tree(tree)
    for i in tree
        println("place\t:",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

0   1
1   0
  1  
  1  
 1 2 
 2 1 
place	:0	rational:	0//1
place	:1	rational:	1//2
place	:2	rational:	1//1
place	:3	rational:	2//1
place	:4	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 2 layers.
  0.000002 seconds
elapsed time (ns):  1770
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [14]:


# 3 Config

function get_depth()
    depth::Int64 = 4
    return depth
end

# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    println("No tree - too big\n")
    #draw_tree(tree)
    for i in tree
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

place:	0	rational:	0//1
place:	1	rational:	1//4
place:	2	rational:	1//3
place:	3	rational:	2//5
place:	4	rational:	1//2
place:	5	rational:	3//5
place:	6	rational:	2//3
place:	7	rational:	3//4
place:	8	rational:	1//1
place:	9	rational:	4//3
place:	10	rational:	3//2
place:	11	rational:	5//3
place:	12	rational:	2//1
place:	13	rational:	5//2
place:	14	rational:	3//1
place:	15	rational:	4//1
place:	16	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 4 layers.
  0.000001 seconds
elapsed time (ns):  1350
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [15]:

# 3 Config

function get_depth()
    depth::Int64 = 8
    return depth
end

# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//8
place:	2	rational:	1//7
place:	3	rational:	2//13
place:	4	rational:	1//6
place:	5	rational:	3//17
place:	6	rational:	2//11
place:	7	rational:	3//16
place:	8	rational:	1//5
place:	9	rational:	4//19
place:	246	rational:	14//3
place:	247	rational:	19//4
place:	248	rational:	5//1
place:	249	rational:	16//3
place:	250	rational:	11//2
place:	251	rational:	17//3
place:	252	rational:	6//1
place:	253	rational:	13//2
place:	254	rational:	7//1
place:	255	rational:	8//1
place:	256	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 8 layers.
  0.000001 seconds
elapsed time (ns):  1410
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [16]:
# 3 Config

function get_depth()
    depth::Int64 = 12
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//12
place:	2	rational:	1//11
place:	3	rational:	2//21
place:	4	rational:	1//10
place:	5	rational:	3//29
place:	6	rational:	2//19
place:	7	rational:	3//28
place:	8	rational:	1//9
place:	9	rational:	4//35
place:	4086	rational:	26//3
place:	4087	rational:	35//4
place:	4088	rational:	9//1
place:	4089	rational:	28//3
place:	4090	rational:	19//2
place:	4091	rational:	29//3
place:	4092	rational:	10//1
place:	4093	rational:	21//2
place:	4094	rational:	11//1
place:	4095	rational:	12//1
place:	4096	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 12 layers.
  0.000002 seconds
elapsed time (ns):  1980
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [21]:
# 3 Config

function get_depth()
    depth::Int64 = 13
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//13
place:	2	rational:	1//12
place:	3	rational:	2//23
place:	4	rational:	1//11
place:	5	rational:	3//32
place:	6	rational:	2//21
place:	7	rational:	3//31
place:	8	rational:	1//10
place:	9	rational:	4//39
place:	8182	rational:	29//3
place:	8183	rational:	39//4
place:	8184	rational:	10//1
place:	8185	rational:	31//3
place:	8186	rational:	21//2
place:	8187	rational:	32//3
place:	8188	rational:	11//1
place:	8189	rational:	23//2
place:	8190	rational:	12//1
place:	8191	rational:	13//1
place:	8192	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 13 layers.
  0.000002 seconds
elapsed time (ns):  1670
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [17]:
# 3 Config

function get_depth()
    depth::Int64 = 14
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//14
place:	2	rational:	1//13
place:	3	rational:	2//25
place:	4	rational:	1//12
place:	5	rational:	3//35
place:	6	rational:	2//23
place:	7	rational:	3//34
place:	8	rational:	1//11
place:	9	rational:	4//43
place:	16374	rational:	32//3
place:	16375	rational:	43//4
place:	16376	rational:	11//1
place:	16377	rational:	34//3
place:	16378	rational:	23//2
place:	16379	rational:	35//3
place:	16380	rational:	12//1
place:	16381	rational:	25//2
place:	16382	rational:	13//1
place:	16383	rational:	14//1
place:	16384	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 14 layers.
  0.000002 seconds
elapsed time (ns):  2030
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [20]:
# 3 Config

function get_depth()
    depth::Int64 = 15
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//15
place:	2	rational:	1//14
place:	3	rational:	2//27
place:	4	rational:	1//13
place:	5	rational:	3//38
place:	6	rational:	2//25
place:	7	rational:	3//37
place:	8	rational:	1//12
place:	9	rational:	4//47
place:	32758	rational:	35//3
place:	32759	rational:	47//4
place:	32760	rational:	12//1
place:	32761	rational:	37//3
place:	32762	rational:	25//2
place:	32763	rational:	38//3
place:	32764	rational:	13//1
place:	32765	rational:	27//2
place:	32766	rational:	14//1
place:	32767	rational:	15//1
place:	32768	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 15 layers.
  0.000002 seconds
elapsed time (ns):  2150
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [22]:
# 3 Config

function get_depth()
    depth::Int64 = 16
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//16
place:	2	rational:	1//15
place:	3	rational:	2//29
place:	4	rational:	1//14
place:	5	rational:	3//41
place:	6	rational:	2//27
place:	7	rational:	3//40
place:	8	rational:	1//13
place:	9	rational:	4//51
place:	65526	rational:	38//3
place:	65527	rational:	51//4
place:	65528	rational:	13//1
place:	65529	rational:	40//3
place:	65530	rational:	27//2
place:	65531	rational:	41//3
place:	65532	rational:	14//1
place:	65533	rational:	29//2
place:	65534	rational:	15//1
place:	65535	rational:	16//1
place:	65536	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 16 layers.
  0.000002 seconds
elapsed time (ns):  2220
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [23]:
# 3 Config

function get_depth()
    depth::Int64 = 24
    return depth
end

    
# 6 control

function main_sb()
    depth       = get_depth()
    tree        = build_tree(depth)
    
    
    println("No tree - too big\n")
    #draw_tree(tree)
    
    println("get too big to print before slowing down")
    for i in tree[1:10]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    
    for i in tree[end-10:end]
        println("place:\t",i.place,"\trational:\t", Rational(i.numerator,i.denominator))
    end
    println("\nUsing the native timing function my Julia Stern-Brocot implementation takes this long.")
    depth           = get_depth()
    
    println("At a depth of ", depth, " layers.")
    @timev @eval build_tree
end

main_sb();

No tree - too big

get too big to print before slowing down
place:	0	rational:	0//1
place:	1	rational:	1//24
place:	2	rational:	1//23
place:	3	rational:	2//45
place:	4	rational:	1//22
place:	5	rational:	3//65
place:	6	rational:	2//43
place:	7	rational:	3//64
place:	8	rational:	1//21
place:	9	rational:	4//83
place:	16777206	rational:	62//3
place:	16777207	rational:	83//4
place:	16777208	rational:	21//1
place:	16777209	rational:	64//3
place:	16777210	rational:	43//2
place:	16777211	rational:	65//3
place:	16777212	rational:	22//1
place:	16777213	rational:	45//2
place:	16777214	rational:	23//1
place:	16777215	rational:	24//1
place:	16777216	rational:	1//0

Using the native timing function my Julia Stern-Brocot implementation takes this long.
At a depth of 24 layers.
  0.000002 seconds
elapsed time (ns):  2310
gc time (ns):       0
bytes allocated:    0
pool allocs:        0
non-pool GC allocs: 0
minor collections:  0
full collections:   0


In [19]:
function test_count_nodes()
    depth = 0
    node_count = count_nodes(depth)    
    @test (node_count == 0) 
    
    depth = 1
    node_count = count_nodes(depth)
    @test (node_count == 1) 
    
    depth = 2
    node_count = count_nodes(depth)
    @test (node_count == 3) 
    
    depth = 10
    node_count = count_nodes(depth)
    @test (node_count == 1023) 
    
    depth = 21
    node_count = count_nodes(depth)
    @test (node_count == 2097151)
    
    println("passed test count nodes")
end

function test_init_tree()
    depth::Int64            = 1
    node_count::Int64       = count_nodes(depth)
    powers::Array{Int64}    = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)
    
    @test child_places == [1]
    @test length(tree) == 3
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[2].numerator    == 1
    @test tree[2].denominator  == 1
    @test tree[2].place        == 1
    @test tree[2].layer        == 1
    
    @test tree[3].numerator    == 1
    @test tree[3].denominator  == 0
    @test tree[3].place        == 2
    @test tree[3].layer        == 0
    
    depth                   = 2
    node_count              = count_nodes(depth)
    powers                  = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)

    @test child_places == [2]
    @test length(tree) == 5
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[3].numerator    == 1
    @test tree[3].denominator  == 1
    @test tree[3].place        == 2
    @test tree[3].layer        == 1
    
    @test tree[5].numerator    == 1
    @test tree[5].denominator  == 0
    @test tree[5].place        == 4
    @test tree[5].layer        == 0
    
    depth                   = 5
    node_count              = count_nodes(depth)
    powers                  = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)

    @test child_places == [16]
    @test length(tree) == 33
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[17].numerator    == 1
    @test tree[17].denominator  == 1
    @test tree[17].place        == 16
    @test tree[17].layer        == 1
    
    @test tree[33].numerator    == 1
    @test tree[33].denominator  == 0
    @test tree[33].place        == 32
    @test tree[33].layer        == 0
    
    println("passed init tree")
end


function JuliaSternBrocotTests()
    test_count_nodes()
    test_init_tree()
    print("passed all tests")
end

JuliaSternBrocotTests()

passed test count nodes
passed init tree
passed all tests